### Kuyesera AI Disaster Damage and Displacement Challenge
* The goal of this notebook is to perform all the infeerence stages of our pipeline
* The notebook assumes that you have read the readme provided and you understand the methodology we used
* To make a brief recap, Our methodology follows a two stages approach
* The two stages include object detection where we detect buildings from pre-images
* Then use the predicted bounding boxes to create patches on both pre and post images
* Then use this pair to perform damaage classification
* We then can train a classifier for different classes. We found that training classifiers for `minor_damage` and `major_damage` were not accurate enough with AUC scores of ~0.8, and the vast distribution shift between xview2 and test data. We therefore only considered a classifier for the `destroyed` class which had an AUC score of ~0.93 in training. We used a Siamese neural network where the pre- and post-disaster images share the same weights. 
* Lastly based on this rule provided on the prizes section on the competition info page:
    * "You must use at least one dataset from the Amazon Sustainability Data Initiative (ASDI) platform as part of your model." We used ndvi values provided by the ASDI platform in our classifier as we assumed that for us to be eligible for the prizes we must follow that rule.

#### Objective
* Since we have provided the diifferent weights and the generated datasets, This notebook makes use of that to run the whole inference pipeline
* If you would like to perform the whole training process then please refer back to the readme to get the steps to do that
  

#### FIRST STAGE: OBJECT DETECTION
* predict only one class : Building from the Malawi pre-disaster Images
* Get the Bounding Boxes (predicted) and use them to crop both the pre-disaster images and post-dissaster image as we neeed the pair for classification

NOTE: We ran this inference notebook on kaggle . If you would like to run it on colab make sure you change kaggle paths to colab. Training on the other hand you need a V100 gpu or any other equivalent (more info on the readme)

In [1]:
!pip install ultralytics -q
!pip install ensemble-boxes -q
!pip install opencv-python -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 13.6 MB/s eta 0:00:00a 0:00:01


In [2]:
from ultralytics import YOLO, RTDETR
import os
import pdb
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import pickle
import sys
from ensemble_boxes import *

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
AUGMENT = True
RUN = 19 # 19
RUN2 = 19
loc = f'runs/detect/train{RUN}'
loc2 = f'runs/detect/train{RUN2}'
SZ = (928, 448)
SZ2 = (928, 448) # 928

THRESH = 0.5
IOU_THRESH = 0.2
CONF = 0.5 # 0.5
YOLO_IOU_THR = 0.35
EXT = 'png'

#using the same model for both paths
model_path = '/kaggle/input/kuyesera-final-submission/Final_sub/yolo_run19/train19/weights/best.pt'
model_path2 = '/kaggle/input/kuyesera-final-submission/Final_sub/yolo_run19/train19/weights/best.pt'
crops_dir = '/kaggle/working/crops'
os.makedirs(crops_dir, exist_ok=True)
test_dir  = "/kaggle/input/kuyesera-final-submission/Final_sub/test_images/Images" #we generated the png equivalents of the provided tif images
tif_dir = "/kaggle/input/kuyesera-final-submission/Final_sub/test_images/Images" #the test images from the zindi data tab
imgs = [os.path.join(test_dir, f) for f in os.listdir(test_dir) if f.endswith(f'_pre_disaster.{EXT}')]
imgs_post = [f.replace('_pre_disaster', '_post_disaster') for f in imgs]
tifs = [os.path.join(tif_dir, f) for f in os.listdir(tif_dir) if f.endswith(f'_pre_disaster.{EXT}')]

def get_areas(boxes):
    areas = []
    for b in boxes:
        arr = abs(b[2] - b[0]) * abs(b[3] - b[1])
        areas.append(arr)
    return areas

def get_results(res, threshold=0.5):
    res     = res.cpu().numpy()
    cls     = res.boxes.cls
    conf    = res.boxes.conf
    boxes   = res.boxes.xyxyn#.tolist()
    mask    = conf > threshold
    conf = conf[mask]
    cls = cls[mask]
    boxes = boxes[mask]

    return cls, boxes, conf

def run_nms(bboxes, confs,labels, image_size, iou_thr=0.50, skip_box_thr=0.0001, weights=None):
    boxes =  [bbox for bbox in bboxes]
    scores = [conf for conf in confs]
    #labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = nms(boxes, scores, labels, weights=weights, iou_thr=iou_thr)
    #boxes = boxes*(image_size-1)
    return boxes, scores, labels

def run_nmw(bboxes, confs,labels, image_size, iou_thr=0.50, skip_box_thr=0.0001, weights=None):
    boxes =  [bbox for bbox in bboxes]
    scores = [conf for conf in confs]
    #labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = non_maximum_weighted(boxes, scores, labels, weights=weights, iou_thr=iou_thr)
    #boxes = boxes*(image_size-1)
    return boxes, scores, labels

def run_soft_nms(bboxes, confs,labels, image_size, iou_thr=0.50, skip_box_thr=0.0001,sigma=0.1, weights=None):
    boxes =  [bbox for bbox in bboxes]
    scores = [conf for conf in confs]
    #labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = soft_nms(boxes, scores, labels, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
    #boxes = boxes*(image_size-1)
    return boxes, scores, labels

def run_wbf(bboxes, confs,labels, image_size, iou_thr=0.50, skip_box_thr=0.0001, weights=None):
    #boxes =  [bbox/(image_size-1) for bbox in bboxes]
    boxes =  [bbox for bbox in bboxes]
    scores = [conf for conf in confs]
    #labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    #boxes = boxes*(image_size-1)
    return boxes, scores, labels


idx = 0
print(imgs[idx])
print(imgs_post[idx])
model = YOLO(model_path)
model2 = YOLO(model_path2)
results = model(imgs[idx], imgsz=SZ, augment=AUGMENT, conf=CONF)
img = results[0].plot()
IM = Image.fromarray(img)
IM.save('yolo.jpg')

results = model(imgs_post[idx], imgsz=SZ, augment=AUGMENT, conf=CONF)
img = results[0].plot()
IM = Image.fromarray(img)
IM.save('yolo_post.jpg')

#sys.exit(0)
img_files = []
preds = []
rep_str = {
        1: "_X_no_damage",
        2: "_X_minor_damage",
        3: "_X_major_damage",
        4: "_X_destroyed",
    }
os.makedirs("crops", exist_ok=True)
data = {}
for im in imgs:
    print(".", end="", flush=True)
    id = os.path.basename(im).replace(f'_pre_disaster.{EXT}', '')
    data[id] = {'pre':[], 'post':[]}
    im2 = im.replace('_pre_disaster', '_post_disaster')
    img_pre = cv2.imread(im)
    img_post = cv2.imread(im2)
    results_pre = model(im, imgsz=SZ, augment=AUGMENT, conf=CONF, iou=YOLO_IOU_THR)
    results_pre2 = model2(im, imgsz=SZ2, augment=AUGMENT, conf=CONF, iou=YOLO_IOU_THR)
    #results_post = model(im2, imgsz=SZ, augment=AUGMENT, conf=0.5)
    #results_post2 = model2(im2, imgsz=SZ, augment=AUGMENT, conf=0.5)
    for res_pre, res_pre2 in zip(results_pre, results_pre2):
        ccls_pre, bbxs_pre, cnfs_pre = get_results(res_pre, threshold=THRESH)
        ccls_pre2, bbxs_pre2, cnfs_pre2 = get_results(res_pre2, threshold=THRESH)
        boxes_pre, scores_pre, labels_pre = run_wbf(
                                     [bbxs_pre, bbxs_pre2], [cnfs_pre, cnfs_pre2],
                                     [ccls_pre, ccls_pre2],
                                     SZ[0], iou_thr=IOU_THRESH)
        #boxes_pre, scores_pre, labels_pre = run_wbf(
        #                             [bbxs_pre], [cnfs_pre], [ccls_pre],
        #                             SZ[0], iou_thr=IOU_THRESH)
        boxes_pre[:,0] = boxes_pre[:,0] * res_pre.orig_shape[1]
        boxes_pre[:,1] = boxes_pre[:,1] * res_pre.orig_shape[0]
        boxes_pre[:,2] = boxes_pre[:,2] * res_pre.orig_shape[1]
        boxes_pre[:,3] = boxes_pre[:,3] * res_pre.orig_shape[0]
        for i in range(len(boxes_pre)):
            x1,y1,x2,y2 = boxes_pre[i]
            crop_pre = img_pre[int(y1):int(y2), int(x1):int(x2)]
            crop_post = img_post[int(y1):int(y2), int(x1):int(x2)]
            if crop_pre.shape[0] != crop_post.shape[0]:
                height = min(crop_pre.shape[0], crop_post.shape[0])
                crop_pre = cv2.resize(crop_pre, (crop_pre.shape[1], height))
                crop_post = cv2.resize(crop_post, (crop_post.shape[1], height))
            # Stitch images side by side
            #stitched_image = np.hstack((crop_pre, crop_post))
            #cv2.imwrite(f"crops/{id}_X_{i}.jpg", stitched_image)
            #data[id]['pre'].append(f"{id}_X_{i}.jpg")
            cv2.imwrite(f"/kaggle/working/crops/{id}_X_{i}_pre.jpg", crop_pre)
            cv2.imwrite(f"/kaggle/working/crops/{id}_X_{i}_post.jpg", crop_post)
            data[id]['pre'].append(f"{id}_X_{i}_pre.jpg")
            data[id]['post'].append(f"{id}_X_{i}_post.jpg")

with open('/kaggle/working/data.pkl', 'wb') as f:
    pickle.dump(data, f)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
/kaggle/input/kuyesera-final-submission/Final_sub/test_images/Images/malawi-cyclone_00000084_pre_disaster.png
/kaggle/input/kuyesera-final-submission/Final_sub/test_images/Images/malawi-cyclone_00000084_post_disaster.png

image 1/1 /kaggle/input/kuyesera-final-submission/Final_sub/test_images/Images/malawi-cyclone_00000084_pre_disaster.png: 224x448 66 buildings, 290.9ms
Speed: 11.5ms preprocess, 290.9ms inference, 275.6ms postprocess per image at shape (1, 3, 224, 448)

image 1/1 /kaggle/input/kuyesera-final-submission/Final_sub/test_images/Images/malawi-cyclone_00000084_post_disaster.png: 224x448 60 buildings, 56.8ms
Speed: 1.0ms preprocess, 56.8ms inference, 1.3ms postprocess pe

### SECOND STAGE: PAIR CLASSIFICATION
* With the image pair from the previous stage and the ndvi values from the ASDI platform(to see how we generated the NDVI values look at the gen_data.py script provided) we now classify the type of damage present.
* Just as explained above, We then can train a classifier for different classes. We found that training classifiers for minor_damage and major_damage were not accurate enough with AUC scores of ~0.8, and the vast distribution shift between xview2 and test data. We therefore only considered a classifier for the destroyed class which had an AUC score of ~0.93 in training. We used a Siamese neural network where the pre- and post-disaster images share the same weights.

In [ ]:
import os
import cv2
import pickle
from timm import create_model
import pdb
import torch
import torch.nn as nn
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import pandas as pd

SZ = 128
FOLD = 0
MODEL = 'efficientvit_b0.r224_in1k'

DATA_DIR = "/kaggle/working/"
CROPS_DIR = "crops" # "mmde_crops" for mmdet, "crops" for yolo
disaster_dir = os.path.join(DATA_DIR, CROPS_DIR)
DATA_FILE = "/kaggle/working/data.pkl" # "mmdet_data.pkl" for mmdet, data.pkl for yolo
with open(DATA_FILE, "rb") as f:
    data = pickle.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_df = pd.read_csv("/kaggle/input/kuyesera-final-submission/Final_sub/test_ndvi.csv")

class DisasterClassifier(nn.Module):
    def __init__(self):
        super(DisasterClassifier, self).__init__()
        self.features = create_model(MODEL, pretrained=True, num_classes=0)

        self.fc = nn.Linear(1+1, 1)

    def euclidean_distance(self, x1, x2):
        return torch.sqrt(torch.sum((x1 - x2)**2, dim=1, keepdim=True))

    def forward(self, pre_image, post_image, x):
        # Extract features from both images
        pre_features = self.features(pre_image)
        post_features = self.features(post_image)

        distance = self.euclidean_distance(pre_features, post_features)
        distance = torch.cat([distance, x], dim=1)
        out = torch.sigmoid(self.fc(distance))
        return out

valid_transform = A.Compose([
    A.Resize(SZ, SZ),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
], additional_targets={'image2': 'image'})


model = DisasterClassifier()
model.load_state_dict(torch.load("/kaggle/input/kuyesera-final-submission/Final_sub/efficientvit_b0.r224_in1k_3_patch_model_fold_0.pth"))
model = model.to(device)
model.eval()
ids = []
damage = []
d_count = 0
for i in data.keys():
    print("[INFO] - Processing", i, flush=True)
    d = {
        0: {'id':f"{i}_X_no_damage", 'count':0},
        1: {'id':f"{i}_X_minor_damage", 'count':0},
        2: {'id':f"{i}_X_major_damage", 'count':0},
        3: {'id':f"{i}_X_destroyed", 'count':0}
    }
    nvdi = torch.tensor(test_df[test_df['id'] == i]['NDVI_mean'].values[0], dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)
    for j in data[i]['pre']:
        pre_image = cv2.imread(os.path.join(disaster_dir, j))
        post_image = cv2.imread(os.path.join(disaster_dir, j.replace("_pre.jpg", "_post.jpg")))
        pre_size = os.path.getsize(os.path.join(disaster_dir, j))
        post_size = os.path.getsize(os.path.join(disaster_dir, j.replace("_pre.jpg", "_post.jpg")))

        #if pre_size < 1024*2 or post_size < 1024*2:
            #if pre_size/post_size >= 2 or post_size < pre_size:
            #    d[3]['count'] += 1
            #elif post_size > pre_size:
            #    d[0]['count'] += 1
        #    continue

        pre_image = cv2.cvtColor(pre_image, cv2.COLOR_BGR2RGB)
        post_image = cv2.cvtColor(post_image, cv2.COLOR_BGR2RGB)
        pre_image = valid_transform(image=pre_image)['image']
        post_image = valid_transform(image=post_image)['image']
        pre_image_ = pre_image.unsqueeze(0).to(device)
        post_image_ = post_image.unsqueeze(0).to(device)
        with torch.no_grad():
            out = model(pre_image_, post_image_, nvdi)
        if out.item() > 0.5:
            d[3]['count'] += 1
            d_count += 1
            pdb.set_trace()
            #cv2.imwrite("abs_pre_image.png", pre_image.permute(1,2,0).cpu().numpy())
            #cv2.imwrite("abs_post_image.png", post_image.permute(1,2,0).cpu().numpy())
            #db.set_trace()
        else:
            d[0]['count'] += 1

    for k in d.keys():
        ids.append(d[k]['id'])
        damage.append(d[k]['count'])

submission = pd.DataFrame({'id':ids, 'damage':damage})
submission.to_csv(f"cls_{MODEL}_submission.csv", index=False)
print("[INFO] - Num destroyed", d_count, flush=True)


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.2 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


model.safetensors:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

[INFO] - Processing malawi-cyclone_00000084
[INFO] - Processing malawi-cyclone_00000241
[INFO] - Processing malawi-cyclone_00000339
[INFO] - Processing malawi-cyclone_00000071
[INFO] - Processing malawi-cyclone_00000254
[INFO] - Processing malawi-cyclone_00000135
[INFO] - Processing malawi-cyclone_00000259
[INFO] - Processing malawi-cyclone_00000082
[INFO] - Processing malawi-cyclone_00000155
[INFO] - Processing malawi-cyclone_00000208
[INFO] - Processing malawi-cyclone_00000026
[INFO] - Processing malawi-cyclone_00000023
[INFO] - Processing malawi-cyclone_00000287
[INFO] - Processing malawi-cyclone_00000263
[INFO] - Processing malawi-cyclone_00000069
[INFO] - Processing malawi-cyclone_00000294
[INFO] - Processing malawi-cyclone_00000327
[INFO] - Processing malawi-cyclone_00000024
[INFO] - Processing malawi-cyclone_00000178
[INFO] - Processing malawi-cyclone_00000196
[INFO] - Processing malawi-cyclone_00000031
[INFO] - Processing malawi-cyclone_00000068
[INFO] - Processing malawi-cyclo